In [1]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
test_df = pd.read_csv("test.csv", encoding="utf-8")

In [2]:
train_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [3]:
med = train_df.median()
train_df = train_df.fillna(med)
test_df = test_df.fillna(med)

In [4]:
most = train_df["Embarked"].value_counts().idxmax()
train_df["Embarked"] = train_df["Embarked"].fillna(most)
test_df["Embarked"] = test_df["Embarked"].fillna(most)

In [5]:
n = "Braund, Mr. Owen Harris"
n = n.split(",")[-1].split(".")[0]
n.strip()

'Mr'

In [6]:
def nameflow(n):
    n = n.split(",")[-1].split(".")[0]
    return n.strip()
mid = train_df["Name"].apply(nameflow).value_counts()
reserved = mid[mid > 30].index
reserved

Index(['Mr', 'Miss', 'Mrs', 'Master'], dtype='object')

In [7]:
def nameflow2(n, r):
    n = n.split(",")[-1].split(".")[0]
    n = n.strip()
    if n in r:
        return n
    else:
        return None
train_df["Name"] = train_df["Name"].apply(nameflow2, args=(reserved, ))
test_df["Name"] = test_df["Name"].apply(nameflow2, args=(reserved, ))

In [8]:
x_train_nodrop = pd.get_dummies(train_df, 
                                columns=["Name", "Sex", "Embarked"])
x_test_nodrop = pd.get_dummies(test_df, 
                                columns=["Name", "Sex", "Embarked"])

In [9]:
x_train = x_train_nodrop.drop(["PassengerId", "Survived", "Cabin", "Ticket"], 
                               axis=1)
y_train = x_train_nodrop["Survived"]
x_test = x_test_nodrop.drop(["PassengerId", "Cabin", "Ticket"], 
                               axis=1)
testid = x_test_nodrop["PassengerId"]

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
params = {
    "n_estimators":range(25, 40),
    "max_depth":range(6, 11)
}
clf = RandomForestClassifier()
cv = GridSearchCV(clf, params, cv=10, n_jobs=4)
cv.fit(x_train, y_train)
print(cv.best_params_)
print(cv.best_score_)

{'max_depth': 7, 'n_estimators': 28}
0.838414481897628


In [52]:
import numpy as np
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=26, max_depth=8)
score = cross_val_score(clf, x_train, y_train, cv=10, n_jobs=4)
print(score)
print(np.average(score))

[0.81111111 0.86516854 0.71910112 0.86516854 0.85393258 0.84269663
 0.84269663 0.78651685 0.85393258 0.84269663]
0.8283021223470662


In [62]:
clf = RandomForestClassifier(n_estimators=26, max_depth=8)
clf.fit(x_train, y_train)
pre = clf.predict(x_test)
result_df = pd.DataFrame({
    "PassengerId":testid,
    "Survived":pre
})
result_df.to_csv("rf.csv", index=False, encoding="utf-8")
result_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [64]:
pd.DataFrame({
    "column":x_train.columns,
    "importance":clf.feature_importances_
})

,column,importance
0,Pclass,0.097505
1,Age,0.133838
2,SibSp,0.062936
3,Parch,0.033885
4,Fare,0.148555
5,Name_Master,0.013326
6,Name_Miss,0.027377
7,Name_Mr,0.196683
8,Name_Mrs,0.038692
9,Sex_female,0.113870


In [57]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train_scale = scaler.fit_transform(x_train)
x_test_scale = scaler.transform(x_test)

In [60]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()
params = {
    "n_neighbors":range(5, 100)
}
cv = GridSearchCV(clf, params, cv=10, n_jobs=4)
cv.fit(x_train_scale, y_train)
print(cv.best_params_)
print(cv.best_score_)

{'n_neighbors': 22}
0.8193508114856428


In [61]:
pre = cv.best_estimator_.predict(x_test_scale)
result_df = pd.DataFrame({
    "PassengerId":testid,
    "Survived":pre
})
result_df.to_csv("knn.csv", index=False, encoding="utf-8")
result_df

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
